##### Импорт библиотек и аутентификация:

In [1]:
# нужные библиотеки
import requests
from requests_ntlm import HttpNtlmAuth
import smtplib
import time
import warnings
warnings.simplefilter(action = 'ignore')
import keyring
from dateutil.relativedelta import relativedelta
from itertools import product
import base64
import os
import pandas as pd
from IPython.display import display

Аутентификация:

In [2]:
auth = HttpNtlmAuth("Login", "Password")

##### Вывод основной информации об отчетах в выбранной папке:

**Инструкция:**
   * По ссылке http://server//reports/api/v2.0/Folders(id)/CatalogItems находится вся информация об отчетах в RS (прод папка). В функции также можно указать id = id нужной папки для получения всей информации об отчетах. Для этого необходимы права на доступ ко всем отчетам в проде.
   * Указать в функции Id своей папки
   * Если папок несколько и необходим общий датасет, то требуется выполнить конкатенацию. Для этого необходимо сложить в список все фреймы и выполнить метод concat: 
df_concat = [df1,df2,df3]
df_concat = pd.concat(df_concat)
   * Если необходимо выгрузить данные в csv,то выполняем команду df.to_csv("df.csv",encoding = "WINDOWS-1251") для кириллической кодировки.

In [3]:
def folder_report_info(folder_id):
    url = f'http://server//reports/api/v2.0/Folders({folder_id})/CatalogItems'
    response = requests.get(url, auth=auth, verify=False)
    resp_json = response.json()
    df_folder = pd.DataFrame(resp_json['value'])
    df_folder['Folder'] = (df_folder["Path"]
                                         .str.split('/', 3).str[2])
    return df_folder

In [5]:
df_1 = folder_report_info('ID_Folder') #Id папки PBI RS
df_2 = folder_report_info('ID_Folder') #Id папки PBI RS
df_3 = folder_report_info('ID_Folder') #Id папки PBI RS
df_info_ = [df_1,df_2,df_3] #Объединение в единый датафрейм
df_info_ = pd.concat(df_info_) #Объединение в единый датафрейм

df_info_.head(2) #посмотреть таблицу с основной информацией по отчетам
#df_info_.to_csv('df_info_.csv',encoding = "WINDOWS-1251") #сохранить результат в csv локально

,@odata.type,Id,Name,Description,Path,Type,Hidden,Size,ModifiedBy,ModifiedDate,CreatedBy,CreatedDate,ParentFolderId,IsFavorite,ContentType,Content,HasDataSources,Roles,Folder
0,#Model.PowerBIReport,98826223-4972-4c1d-a9a7-0729a1451dad,Cвязь продуктивность,None,/Отчеты Power BI/СПОК - Ключевые метрики/Cвязь...,PowerBIReport,False,38990195,RU1000\60127133,2023-06-06T11:07:39.44+03:00,RU1000\60127133,2023-06-06T11:07:39.44+03:00,c088bf42-0ee6-4ea4-b46e-909f9c6d9f63,False,None,,True,[],СПОК - Ключевые метрики
1,#Model.PowerBIReport,d9a03c0f-26c5-4667-907c-9910339fa38a,Доля чеков по специальностям,None,/Отчеты Power BI/СПОК - Ключевые метрики/Доля ...,PowerBIReport,False,3142374,RU1000\60127133,2023-06-06T11:08:12.987+03:00,RU1000\60127133,2023-06-06T11:08:12.847+03:00,c088bf42-0ee6-4ea4-b46e-909f9c6d9f63,False,None,,True,[],СПОК - Ключевые метрики


##### Вывод информации о статусах обновлений отчетов в выбранной папке:

Формирование полной ссылки для получения информации об обновлении отчетов:

In [ ]:
http_1 = 'http://server//reports/api/v2.0/PowerBIReports('
http_2 = ')/CacheRefreshPlans'
url = [http_1 + x + http_2 for x in df_info_['Id']] #из предыдущей таблицы с основной информацией об отчетах.
url #посмотреть все ссылки по статусам обновлений отчетов

Выгрузка информации о статусах обновлений выбранных ранее отчетов:

In [7]:
df_info_refresh = [] #объявляем пустой список, в который будем складывать записи по api

#Собираем данные для фрейма
for i in range(len(url)): #цикл по ранее полученным ссылкам на отчеты
    response = requests.get(str(url[i]), auth=auth, verify=False) #подключение по api
    resp_json = response.json() #складываем в json
    df_refresh = pd.DataFrame(resp_json['value']) #складываем в фрейм
    for j in range(len(df_refresh)): ##для чтения всех записей фреймов
        df_info_refresh.append(df_refresh) #добавляем очередную запись в итогоый список
df_info_refresh = pd.concat(df_info_refresh) #склеииваем записи в единый фрейм

#Обработка фрейма
df_info_refresh = ( 
    df_info_refresh[['Id', 'Owner', 'Description', 
                             'CatalogItemPath', 'EventType',
                             'ScheduleDescription', 'LastRunTime', 
                             'LastStatus', 'ModifiedBy','ModifiedDate']]) #объявляем все строки, которые нужны для выгрузки

df_info_refresh = df_info_refresh.drop_duplicates(keep='last') #удаляем дубликаты (не последние записи)
df_info_refresh["ReportName"] = (df_info_refresh["CatalogItemPath"]
                                         .str.split('/', -1).str[-1]) #создаем столбец с названием отчета

df_info_refresh.head(3) #посмотреть таблицу со статусом обновлений
# df_info_refresh.to_csv('df_inforefresh.csv',encoding = "WINDOWS-1251") #сохранить результат в csv локально

,Id,Owner,Description,CatalogItemPath,EventType,ScheduleDescription,LastRunTime,LastStatus,ModifiedBy,ModifiedDate,ReportName
0,d38a6224-d0cb-46e5-bdd0-107a905e265f,RU1000\60127133,Обновление 1,/Отчеты Power BI/СПОК - Ключевые метрики/Доля ...,DataModelRefresh,"At 2:45 AM every Sun, Mon, Tue, Wed, Thu, Fri...",2023-10-30T02:45:39.177+03:00,Completed Data Refresh,RU1000\60206272,2023-10-10T10:43:07.217+03:00,Доля чеков с картой по сотруднику
0,25bae7e7-2fbd-4b5d-b2d7-ba588cc52088,RU1000\60127133,Обновление отчета заменить после на 11:00,/Отчеты Power BI/СПОК - Ключевые метрики/КСО,DataModelRefresh,"At 10:00 PM every day, starting 9/18/2023",2023-10-29T22:07:31.677+03:00,Completed Data Refresh,RU1000\60127133,2023-09-18T10:20:57.75+03:00,КСО
0,7d1cc2f0-7b03-4b69-9c63-bb36c2672f67,RU1000\60127133,Обновление,/Отчеты Power BI/СПОК - Ключевые метрики/Медиа...,DataModelRefresh,"At 3:25 AM every Sun, Mon, Tue, Wed, Thu, Fri...",2023-10-30T03:35:22.393+03:00,"Data Refresh failed, contact the administrator...",RU1000\60004177,2023-10-16T18:54:40.713+03:00,Медианный чек 2.0


##### Отправка результатов о статусах обновлений на почту

In [8]:
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import smtplib
import sys

Формирование таблицы для отправки:

In [9]:
df_failed_refresh = (df_info_refresh[['Owner','ReportName','CatalogItemPath',
                                              'Description','LastRunTime','LastStatus']]
                     [df_info_refresh['LastStatus']!='Completed Data Refresh'])

df_failed_refresh.head(3) #посмотреть таблицу для отправки

,Owner,ReportName,CatalogItemPath,Description,LastRunTime,LastStatus
0,RU1000\60127133,Медианный чек 2.0,/Отчеты Power BI/СПОК - Ключевые метрики/Медиа...,Обновление,2023-10-30T03:35:22.393+03:00,"Data Refresh failed, contact the administrator..."
0,RU1000\60127133,Показатели работы кассиров,/Отчеты Power BI/СПОК - Ключевые метрики/Показ...,Обновление,2023-10-30T04:50:58.303+03:00,"Data Refresh failed, contact the administrator..."
0,RU1000\60138804,Чеки под чужим лдап,/Отчеты Power BI/СПОК - Ключевые метрики/Чеки ...,тест 4,2023-10-30T13:21:16.927+03:00,"Data Refresh failed, contact the administrator..."


ВАЖНО: для авторизации почты FROMADDR необходимо получить код из 16 символов в gmail и указать его в PASSWORD.
https://stackoverflow.com/questions/60701936/error-invalid-login-application-specific-password-required

Отправка письма с табличными данными на почту:

In [10]:
recipients = ['mail1.ru','mail2.ru'] #Список получателей
emaillist = [elem.strip().split(',') for elem in recipients]
msg = MIMEMultipart()
msg['Subject'] = "Daily Failed Refresh PBI Reporting" #Тема письма
FROMADDR = "you@gmail.com" #С какого почтового адреса будет отправка gmail.com
LOGIN    = FROMADDR #Логин/Отправитель
PASSWORD = "yourcode" #полученный 16-значный код


html = """\
<html>
  <head></head>
  <body>
    {0}
  </body>
</html>
""".format(df_failed_refresh.to_html()) #Формирование фрейма для отображения в письме

part1 = MIMEText(html, 'html')
msg.attach(part1)

In [ ]:
server = smtplib.SMTP('smtp.gmail.com', 587) #Отправка письма
server.set_debuglevel(1)
server.set_debuglevel(1)
server.ehlo()
server.starttls()
server.login(LOGIN, PASSWORD)
server.sendmail(FROMADDR, emaillist , msg.as_string())
server.quit() #Закрытие подключения